# **Saisonnalité et durée de la saisonnalité**

## Configuration

In [ ]:
# Set SNT Paths
SNT_ROOT_PATH  <- "~/workspace"
CODE_PATH      <- file.path(SNT_ROOT_PATH, "code")
CONFIG_PATH    <- file.path(SNT_ROOT_PATH, "configuration")

# load util functions
source(file.path(CODE_PATH, "snt_utils.r"))

# List required packages 
required_packages <- c("dplyr", "tidyr", "terra", "ggplot2", "stringr", "lubridate", "viridis", "patchwork", "zoo", "purrr", "arrow", "sf", "reticulate", "leaflet")

# Execute function
install_and_load(required_packages)

# Set environment to load openhexa.sdk from the right environment
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

# Load SNT config
config_json <- tryCatch({ jsonlite::fromJSON(file.path(CONFIG_PATH, "SNT_config.json"))},
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

# Required environment for the sf packages
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")

In [ ]:
# Configuration variables
DATASET_NAME <- config_json$SNT_DATASET_IDENTIFIERS$SNT_SEASONALITY
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
ADM_2 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_2)

In [ ]:
# print function
printdim <- function(df, name = deparse(substitute(df))) {
  cat("Dimensions of", name, ":", nrow(df), "rows x", ncol(df), "columns\n\n")
}

## Saisonnalité estimée sur la base des données pluviométriques

In [ ]:
# import seasonality data
seasonality_data <- tryCatch({ get_latest_dataset_file_in_memory(DATASET_NAME, paste0(COUNTRY_CODE, "_precipitation_seasonality.parquet")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading seasonality file for: " , COUNTRY_CODE, conditionMessage(e))
                      cat(msg)
                      stop(msg)
                      })
# import DHIS2 shapes data
DATASET_DHIS2 <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED
shapes_data <- tryCatch({ get_latest_dataset_file_in_memory(DATASET_DHIS2, paste0(COUNTRY_CODE, "_shapes.geojson")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 Shapes data for: " , COUNTRY_CODE, conditionMessage(e))
                      cat(msg)
                      stop(msg)
                      })

printdim(seasonality_data)
head(seasonality_data)

In [ ]:
# merge polygon file with seasonality dataset
seasonality_data <- shapes_data %>%
  left_join(seasonality_data, by = c("ADM2_ID" = "ADM2_ID"))  # or by ADM2_NAME

## Create plots

In [ ]:
# --- Prepare data ---
seasonality_data <- seasonality_data %>%
  mutate(
    DUREE_CATEGORIE = as.factor(SEASONAL_BLOCK_DURATION_PRECIPITATION),
    SAISONNALITE = ifelse(SEASONALITY_PRECIPITATION == 1, "Saisonnier", "Non saisonnier")
  )

# --- Map 1: Présence de la saisonnalité ---
p1 <- ggplot(seasonality_data) +
  geom_sf(aes(fill = SAISONNALITE), color = "white") +
  scale_fill_manual(values = c("Saisonnier" = "darkgreen")) +
  labs(title = "Présence de la saisonnalité", fill = "Saisonnalité") +
  theme_minimal(base_size = 16) +
  theme(
    plot.title = element_text(size = 20, face = "bold"),
    legend.position = "bottom"
  )

# --- Map 2: Durée de la saisonnalité (catégories) ---
p2 <- ggplot(seasonality_data) +
  geom_sf(aes(fill = DUREE_CATEGORIE), color = "white") +
  scale_fill_manual(
    values = c("3" = "#2b83ba", "4" = "#abdda4", "5" = "#fdae61"),
    name = "Durée (mois)",
    labels = c("3" = "3 mois", "4" = "4 mois", "5" = "5 mois")
  ) +
  labs(title = "Durée de la saisonnalité") +
  theme_minimal(base_size = 16) +
  theme(
    plot.title = element_text(size = 20, face = "bold"),
    legend.position = "bottom"
  )


In [ ]:
# Combine plots
options(repr.plot.width = 18, repr.plot.height = 7)
p1 + p2